# Задание 4*. Посимвольная генерация текста (Solution)

Возьмите произведение Гете "Фауст" и обучите на нем LSTM-модель для посимвольной генерации текста. Вместо one-hot кодирования используйте `nn.Embedding` [🛠️[doc]](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html). При обучении игнорируйте знаки препинания и номера страниц.

[[doc] 🛠️ Word Embeddings Tutorial](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html)

Импорт необходимых библиотек:

In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader

Загрузка данных:

In [ ]:
!wget -q https://edunet.kea.su/repo/EduNet-web_dependencies/datasets/Faust.txt

In [ ]:
with open("Faust.txt") as text_file:
    faust_text = "".join(text_file.readlines())

In [ ]:
# Your code here
class FaustDataset(Dataset):
    def __init__(self, data, seq_len=256, chars="абвгдеёжзийклмнопрстуфхцчшщъыьэюя \n"):
        super().__init__()

        self.seq_len = seq_len
        # Creating a dictionary that maps integers to the characters
        chars = list(chars)
        self.int2char = dict(enumerate(chars))
        # Creating another dictionary that maps characters to integers
        self.char2int = {char: ind for ind, char in self.int2char.items()}

        # encode data
        self.data = self.encode(data)

    def encode(self, char_string):
        return [
            self.char2int[character]
            for character in char_string
            if character in self.char2int
        ]

    def decode(self, int_string):
        return [
            self.int2chars[integer]
            for integer in int_string
            if integer in self.int2chars
        ]

    def __getitem__(self, n):
        patch = self.data[n : n + self.seq_len]
        x = torch.LongTensor(patch[:-1])
        y = torch.LongTensor(patch[1:])
        return x, y

    def __len__(self):
        return len(self.data) - self.seq_len - 1

In [ ]:
train_set = FaustDataset(faust_text)
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
dict_size = len(train_set.int2char)
print(dict_size)

In [ ]:
class TextRNN(nn.Module):
    def __init__(self, dict_size, hidden_size, embedding_size, n_layers):
        super(TextRNN, self).__init__()

        self.dict_size = dict_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(self.dict_size, self.embedding_size)
        self.lstm = nn.LSTM(
            self.embedding_size, self.hidden_size, self.n_layers, batch_first=True
        )
        # self.dropout = nn.Dropout(0.2) # can work without
        self.fc = nn.Linear(self.hidden_size, self.dict_size)

    def forward(self, x, hidden):
        x = self.encoder(x)
        out, (ht1, ct1) = self.lstm(x, hidden)
        out = out.contiguous().view(-1, self.hidden_size)
        # out = self.dropout(out) # can work without
        x = self.fc(out)
        return x, (ht1, ct1)

    def init_hidden(self, batch_size=1):
        return (
            torch.zeros(
                self.n_layers, batch_size, self.hidden_size, requires_grad=True
            ).to(device),
            torch.zeros(
                self.n_layers, batch_size, self.hidden_size, requires_grad=True
            ).to(device),
        )

In [ ]:
def evaluate(model, char2int, int2char, start_text="\n", prediction_len=256):
    hidden = model.init_hidden()
    idx_input = [char2int[char] for char in start_text]
    train = torch.LongTensor(idx_input).view(-1, 1).to(device)
    predicted_text = start_text

    _, hidden = model(train, hidden)

    inp = train[-1].view(-1, 1)

    for i in range(prediction_len):
        output, hidden = model(inp.to(device), hidden)
        output_logits = output.cpu().data.view(-1)
        p_next = F.softmax(output_logits * 3, dim=-1).detach().cpu().data.numpy()
        top_index = np.random.choice(len(char2int), p=p_next)
        inp = torch.LongTensor([top_index]).view(-1, 1).to(device)
        predicted_char = int2char[top_index]
        predicted_text += predicted_char

    return predicted_text

In [ ]:
model = TextRNN(
    dict_size=len(train_set.int2char), hidden_size=128, embedding_size=32, n_layers=3
)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)


num_epochs = 1
loss_avg = []

for epoch in range(num_epochs):
    loss_list = []
    i = 0
    for train, target in tqdm(train_loader):
        model.train()
        hidden = model.init_hidden(train.shape[0])
        train, target = train.to(device), target.to(device)

        output, hidden = model(train, hidden)
        loss = criterion(output, target.view(-1))

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        loss_list.append(loss.item())

        i += 1
        if i % 500 == 0:
            model.eval()
            mean_loss = np.mean(loss_list)
            print(f"Epoch: {epoch + 1}, batch: {i}, Loss: {mean_loss}")
            loss_list = []
            print(evaluate(model, train_set.char2int, train_set.int2char))

In [ ]:
model.eval()
print(evaluate(model, train_set.char2int, train_set.int2char))

## Формат результата

Сгенерерированный текст

Пример текста:

"все все от бесстыдные старой

все в нем получше все стремленья

поддержки с собой в сердце воздух своей

и в вечной страсти восстанет свой предлог

привет вам слуга в сладком страшней стране

и в мире все вражда станет станет

в поле на пользу своим воспоминанья"

## Памятка для преподавателя

1. Укажите, что нужно сделать 2 вывода. Один при разных стартовых буквах, другой — разные фамилии на одну и ту же букву.
2. При использовании `random.choise` с весами-вероятностями могут возникать ошибки округления этих вероятностей. Если такое возникает, можно перейти от этой функции к любой другой (например, брать топ-$k$ вариантов и выбирать из них случайный).
3. Можно предложить сгенерировать этот же текст с помощью трансформера.

# Задание 4*. Посимвольная генерация текста (student version)

Возьмите произведение Гете "Фауст" и обучите на нем LSTM-модель для посимвольной генерации текста. Вместо one-hot кодирования используйте `nn.Embedding` [🛠️[doc]](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html). При обучении игнорируйте знаки препинания и номера страниц.

[[doc] 🛠️ Word Embeddings Tutorial](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html)


Импорт необходимых библиотек:

In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader

Загрузка данных:

In [ ]:
!wget -q https://edunet.kea.su/repo/EduNet-web_dependencies/datasets/Faust.txt

In [ ]:
with open("Faust.txt") as text_file:
    faust_text = "".join(text_file.readlines())

In [ ]:
# Your code here

## Формат результата

Сгенерерированный текст

Пример текста:

"все все от бесстыдные старой

все в нем получше все стремленья

поддержки с собой в сердце воздух своей

и в вечной страсти восстанет свой предлог

привет вам слуга в сладком страшней стране

и в мире все вражда станет станет

в поле на пользу своим воспоминанья"
